In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pymc as pm
import numpy as np
import pandas as pd
from bayesinsight import BayesInsightModel
import yaml
from scipy.optimize import minimize
import xarray as xr

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [4]:
with open("optimizer_setup.yaml", "r") as f:
    settings = yaml.safe_load(f)

In [5]:
START_PERIOD = settings["start_period"]
END_PERIOD = settings["end_period"]
MODELS = settings["models"]
MODEL_NAMES = list(settings["models"].keys())
CHANNEL_TO_VARNAMES_FOOTTRAFFIC = settings["varnames"]["foottraffic"]
VARIABLE_TO_CHANNEL_FOOTTRAFFIC = {
    v: k for k, vs in CHANNEL_TO_VARNAMES_FOOTTRAFFIC.items() for v in vs
}
ORIGINAL_BUDGET = settings["current_budget"]
ORIGINAL_CPM = settings["current_cpm"]
FOOTTRAFFIC_MODEL_PATH = MODELS["foottraffic"]["model_location"]
FOOTTRAFFIC_MODEL_DIMS = MODELS["foottraffic"]["dims"]

In [14]:
model = BayesInsightModel.load(FOOTTRAFFIC_MODEL_PATH)

In [15]:
pymc_model = model.build()

In [16]:
model_trace = model.trace

In [17]:
media_vars = model.return_media_variables()

In [18]:
posterior = model_trace.posterior
model_data = model_trace.constant_data

In [19]:
paid_media = [
    media_var
    for media_var in media_vars
    if media_var.variable_name in VARIABLE_TO_CHANNEL_FOOTTRAFFIC.keys()
]

In [20]:
def objective(mu, dim=("Period", "Geography"), model=model):
    original_model_mu = model.trace.posterior.mu.sum(dim=dim).mean().values
    new_model_mu = mu.sum(dim=dim).mean().values

    return (-new_model_mu + original_model_mu) / original_model_mu

In [21]:
def get_scaling_factor(original_spend, new_spend, original_cpm, new_cpm):
    original_imps = original_spend / original_cpm * 1000
    new_imps = new_spend / new_cpm * 1000
    return new_imps / original_imps

In [22]:
def weights_to_scaling_factors(
    x: list[float],
    new_budgets: dict[str, float],
    new_cpms: dict[str, float],
    original_budget=ORIGINAL_BUDGET,
    original_cpm=ORIGINAL_CPM,
):
    """Scale impressions during the start and end period by
    the fraction more or less impressions the new budget would imply based on
    the old budget and cpms.
    This allows the solver to look through percentage increase/decrease bounds instead of actuall $ amounts.
    """
    scaling_factor = {
        key: get_scaling_factor(
            original_budget[key],
            new_budgets[key] * (1 + x[i]),
            original_cpm[key],
            new_cpms[key],
        )
        - 1
        for i, key in enumerate(original_budget.keys())
    }

    return scaling_factor

In [23]:
def make_input_mask(start_period, end_period, model_dates):
    "Treat periods outside the masked period as if they didn't change"

    if isinstance(start_period, str):
        start_period = pd.to_datetime(start_period)

    if isinstance(end_period, str):
        end_period = pd.to_datetime(end_period)
    input_mask = xr.DataArray(
        np.ones_like(model_dates, dtype=float),
        dims=("Period",),
        coords={"Period": model_dates},
    ).where(lambda x: (x.Period > start_period) & (x.Period < end_period), 0)
    return input_mask

In [42]:
pd.Timestamp

pandas._libs.tslibs.timestamps.Timestamp

In [40]:
type(pd.to_datetime("2023-04-01"))

pandas._libs.tslibs.timestamps.Timestamp

In [24]:
make_input_mask("2023-04-01", "2024-04-01", model.trace.posterior.Period.values)

<xarray.DataArray (Period: 156)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])
Coordinates:
  * Period   (Period) datetime64[ns] 2021-04-05 2021-04-12 ... 2024-03-25

In [25]:
model.trace.constant_data.dims

Frozen({'Geography': 25, 'Period': 156, 'LLT_splines': 16})

In [26]:
def compute_contributions(
    x,
    new_budget,
    new_cpm,
    start_period,
    end_period,
    model,
    var_map=VARIABLE_TO_CHANNEL_FOOTTRAFFIC,
):
    updated_weights = weights_to_scaling_factors(x, new_budget, new_cpm)
    input_mask = make_input_mask(start_period, end_period, model.trace.posterior.Period)
    pymc_model = model.build()
    with pymc_model:
        pm.set_data(
            {
                media_var.variable_name: (
                    (
                        updated_weights[
                            VARIABLE_TO_CHANNEL_FOOTTRAFFIC[media_var.variable_name]
                        ]
                        * input_mask
                        + 1
                    )
                    * model.trace.constant_data[media_var.variable_name]
                ).transpose(*model.trace.constant_data[media_var.variable_name].dims)
                for i, media_var in enumerate(paid_media)
            }
        )
        det_ = pm.compute_deterministics(model.trace.posterior)

    return det_

In [27]:
def non_heirarcical_bayesinsight_loss_function(
    x,
    new_budget,
    new_cpm,
    input_mask,
    external_variables=None,
    chains_to_use=slice(0, 1),
    draws_to_use=slice(0, 1000, 5),
    var_map=VARIABLE_TO_CHANNEL_FOOTTRAFFIC,
    model=model,
):
    updated_weights = weights_to_scaling_factors(x, new_budget, new_cpm)
    pymc_model = model.build()
    with pymc_model:
        pm.set_data(
            {
                media_var.variable_name: (
                    (updated_weights[var_map[media_var.variable_name]] * input_mask + 1)
                    * model.trace.constant_data[media_var.variable_name]
                ).transpose(*model.trace.constant_data[media_var.variable_name].dims)
                for i, media_var in enumerate(paid_media)
            }
        )
        if external_variables is not None:
            pass
        det_ = pm.compute_deterministics(
            model.trace.posterior.sel(chain=chains_to_use, draw=draws_to_use),
            var_names=["mu"],
            progressbar=False,
        )
    return objective(det_.mu)

In [28]:
INPUT_MASK = make_input_mask(START_PERIOD, END_PERIOD, model.trace.posterior.Period)

In [29]:
same_budget = non_heirarcical_bayesinsight_loss_function(
    np.zeros(shape=len(ORIGINAL_BUDGET.keys())),
    ORIGINAL_BUDGET,
    ORIGINAL_CPM,
    INPUT_MASK,
)

In [30]:
same_budget

-0.00012775072240356073

In [31]:
assert np.abs(same_budget) < 1e-3

In [32]:
model.trace.constant_data

<xarray.Dataset>
Dimensions:                                                           (
                                                                       Geography: 25,
                                                                       Period: 156,
                                                                       LLT_splines: 16)
Coordinates:
  * Geography                                                         (Geography) object ...
  * Period                                                            (Period) datetime64[ns] ...
  * LLT_splines                                                       (LLT_splines) int32 ...
Data variables: (12/16)
    Christmas_Day_Total_Total_Total_Total                             (Geography, Period) float64 ...
    Display_Impressions_agg_product_campaign_Total_Total_Total_Total  (Geography, Period) float64 ...
    Foot_Traffic_Total_Total_Total_Total                              (Geography, Period) float64 ...
    LLT_spline_matrix                                                 (Period, LLT_splines) float64 ...
    Magazine_Circulation_agg_product_Total_Total_Total_Total          (Geography, Period) float64 ...
    Newspaper_Circulation_agg_product_Total_Total_Total_Total         (Geography, Period) float64 ...
    ...                                                                ...
    Paid_Search_Clicks_agg_outlet_product_Total_Total_Generic_Total   (Geography, Period) float64 ...
    Paid_Social_Impressions_agg_full_Total_Total_Total_Total          (Geography, Period) float64 ...
    Snow_Total_Total_Total_Total                                      (Geography, Period) float64 ...
    T-Avg_Total_Total_Total_Total                                     (Geography, Period) float64 ...
    Valentine's_Day_Total_Total_Total_Total                           (Geography, Period) float64 ...
    YouGov_Ad_Awareness_Pct_Yes_Total_Total_Total_Total               (Geography, Period) float64 ...
Attributes:
    created_at:                 2024-09-11T17:46:06.615670+00:00
    arviz_version:              0.19.0
    inference_library:          numpyro
    inference_library_version:  0.15.2
    sampling_time:              8918.256319
    tuning_steps:               1000

In [33]:
increased_budget = non_heirarcical_bayesinsight_loss_function(
    0.3 * np.ones(shape=len(paid_media)), ORIGINAL_BUDGET, ORIGINAL_CPM, INPUT_MASK
)

In [34]:
assert (
    increased_budget <= same_budget
)  # Increased budget should have a lower or same loss as the original budget

In [35]:
lower_budget = non_heirarcical_bayesinsight_loss_function(
    -0.3 * np.ones(shape=len(paid_media)), ORIGINAL_BUDGET, ORIGINAL_CPM, INPUT_MASK
)

In [36]:
assert lower_budget >= same_budget  # Lowered budget

In [37]:
weights_to_scaling_factors?

Signature:
weights_to_scaling_factors(
    x: list[float],
    new_budgets: dict[str, float],
    new_cpms: dict[str, float],
    original_budget={'OLV': 3606151, 'Paid Search': 16235250, 'Paid Social': 16977498, 'OOH': 14493428, 'Newspaper': 1520711, 'Display': 9041363, 'Magazine': 6799296},
    original_cpm={'OLV': 13.81, 'Display': 18.97, 'Magazine': 119.25, 'OOH': 14.42, 'Paid Search': 1223.15, 'Paid Social': 6.51, 'Newspaper': 121.41},
)
Docstring:
Scale impressions during the start and end period by 
the fraction more or less impressions the new budget would imply based on 
the old budget and cpms.
This allows the solver to look through percentage increase/decrease bounds instead of actuall $ amounts.
File:      c:\users\matthew.reda\appdata\local\temp\ipykernel_23460\2637627710.py
Type:      function

In [73]:
def eq_constraint(x, starting_budget: dict[str, float]):
    starting_budget_ = sum(starting_budget.values())
    updated_budget = sum(
        (1 + x[i]) * starting_budget[media_var]
        for i, media_var in enumerate(starting_budget.keys())
    )

    return (starting_budget_ - updated_budget) / starting_budget_

In [74]:
sum(ORIGINAL_BUDGET.values()) - TOTAL_BUDGET

0

In [76]:
eq_constraint([0, 0, 0, 0, 0, 0, 0], ORIGINAL_BUDGET)

0.0

In [81]:
sol = minimize(
    non_heirarcical_bayesinsight_loss_function,
    args=(ORIGINAL_BUDGET, ORIGINAL_CPM, INPUT_MASK),
    x0=np.zeros(len(ORIGINAL_BUDGET)),
    bounds=[(-0.3, 0.3)] * len(ORIGINAL_BUDGET),
    constraints=dict(type="eq", fun=lambda x: eq_constraint(x, ORIGINAL_BUDGET)),
)

In [6]:
import pickle
# with open("sol.pkl", 'wb') as f:
#    pickle.dump(sol, f)

In [7]:
with open("sol.pkl", "rb") as f:
    s = pickle.load(f)

In [8]:
s

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.04270176132993113
       x: [ 3.000e-01  3.000e-01 -1.592e-01 -3.000e-01  4.334e-02
           -1.115e-01  3.000e-01]
     nit: 10
     jac: [-3.894e-03 -1.615e-01 -5.117e-03 -1.770e-03 -1.025e-03
           -2.412e-03 -4.046e-03]
    nfev: 80
    njev: 10

In [10]:
new_budget = {
    key: (1 + s.x[i]) * value for i, (key, value) in enumerate(ORIGINAL_BUDGET.items())
}

In [11]:
TOTAL_BUDGET

68673697

In [12]:
diff = {key: (new_budget[key] - value) for key, value in ORIGINAL_BUDGET.items()}
{key: diff[key] for key in sorted(diff, key=lambda x: diff[x])}

{'OOH': -4348028.399999993,
 'Paid Social': -2702346.481288908,
 'Display': -1007745.7104542628,
 'Newspaper': 65911.49174317648,
 'OLV': 1081845.2999999989,
 'Magazine': 2039788.7999999989,
 'Paid Search': 4870574.999999996}

In [38]:
solution_contributions = compute_contributions(
    s.x,
    ORIGINAL_BUDGET,
    ORIGINAL_CPM,
    start_period=START_PERIOD,
    end_period=END_PERIOD,
    model=model,
)

Output()